[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ofz5Ep1Pnti7kPzYe0dRqxMHxU2pDCvH#scrollTo=BSF-vb-9MWAz)


### Step 1: Install Dependencies

This cell uses the `%pip` magic command to install the `flotorch` library. The `[adk]` extra specifies that all necessary dependencies for the Agent Development Kit (ADK), such as `google-adk`, should also be installed. This package provides the core tools for building and running your agent.

In [ ]:
%pip install flotorch[adk]

### Step 2: Configuration Variables

This cell defines the constant variables required to configure and authenticate the Flotorch agent.

* `FLOTORCH_API_KEY`: Replace the API key with your actual FloTorch API Key (Check [Prerequisites.ipynb](https://github.com/FloTorch/Resources/blob/main/examples/flotorch-rag-notebooks/faiss-example/01_Prerequisites.ipynb) on how to get the key)
                       
                       Using FloTorch helps you monitor the agentic flow through the FloTorch gateway.
* `FLOTORCH_BASE_URL`: The API gateway endpoint.
* `AGENT_NAME`: A unique name for this specific agent instance, indicating it uses MCP (Multi-turn Conversation Protocol) tools.
* `APP_NAME`: A name for your application, used to group sessions.
* `USER_ID`: A unique identifier for the end-user interacting with the agent.

In [ ]:
from getpass import getpass

FLOTORCH_API_KEY = getpass("FloTorch API Key") # SET YOUR FLOTORCH API KEY HERE
FLOTORCH_BASE_URL = "https://gateway.flotorch.cloud"
AGENT_NAME = "crypto-agent"
APP_NAME = "flotorch_tools_example"
USER_ID = "flotorch_user_001"

### Step 3: Import Libraries

This cell imports the core classes needed from the Flotorch ADK and Google ADK libraries:
* `FlotorchADKAgent`: For creating the agent instance connected to the Flotorch platform.
* `FlotorchADKSession`: For managing conversation sessions using Flotorch's session service.
* `Runner`: The ADK component that orchestrates the agent's execution flow.
* `types` (from `google.genai`): Provides standard data structures like `Content` and `Part` for messages.

In [ ]:
from flotorch.adk.agent import FlotorchADKAgent
from flotorch.adk.sessions import FlotorchADKSession
from google.adk import Runner
from google.genai import types

print("Imported necessary libraries successfully")

### Step 4: Initialize the Flotorch Agent

Here, an instance of `FlotorchADKAgent` is created. It's configured with the agent's unique name (`AGENT_NAME`), API credentials (`FLOTORCH_API_KEY`, `FLOTORCH_BASE_URL`). This client object will automatically fetch the agent's configuration, including any tools defined on the Flotorch platform (like the MCP tools this agent is designed to use), when `.get_agent()` is called.

In [ ]:
flotorch_client =  FlotorchADKAgent(
        agent_name=AGENT_NAME,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_BASE_URL
    )


agent = flotorch_client.get_agent()
print(f"FlotorchADKAgent '{agent.name}' created successfully.")

### Step 5: Initialize Session Service and Runner

This cell sets up the remaining components needed to run the agent:
* `FlotorchADKSession`: Initializes the session service using the Flotorch backend to store and retrieve conversation history.
* `Runner`: Initializes the ADK Runner, connecting the initialized `agent` with the `session_service`. The `APP_NAME` is used to logically group sessions under this application.

In [ ]:
session_service = FlotorchADKSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

runner = Runner(
        agent=agent,
        app_name=APP_NAME,
        session_service=session_service
    )

print("Runner configured successfully for FlotorchADKAgent.")

### Step 6: Define Chat Helper Functions

This cell defines two functions to manage the interaction with the agent:
* `run_single_turn`: Takes a query, session ID, user ID, and the runner. It formats the query, sends it to the runner, processes the resulting events, and extracts/returns the agent's final text response.
* `chat_with_agent`: An asynchronous wrapper around `run_single_turn` that simplifies the interface for sending a query and getting a response within a specific session.

In [ ]:
def run_single_turn(query, session_id, user_id, runner):
    """
    Send a message to the FlotorchADKAgent and get the response.
    """
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text

    return "Sorry, I couldn't process that request."

async def chat_with_agent(query,session_id):
    """
    Chat function that takes a user query and returns the FlotorchADKAgent's response.
    """
    response = run_single_turn(query, session_id, USER_ID, runner)
    return response

print("Chat function ready. Use chat_with_agent('your query') to interact with the FlotorchADKAgent.")

### Step 7: Run Example Interaction

This cell demonstrates how to use the agent configured with an MCP tool.

1.  It first creates a new session for the specified `APP_NAME` and `USER_ID`.
2.  It then calls `chat_with_agent` with a query. In the example provided ("explain about this repo facebook/react"), the agent might interpret this differently, but if a query related to cryptocurrency prices or data (e.g., "What's the price of Bitcoin?") were used, the following would happen:
3.  The agent, configured on the Flotorch platform with MCP tools pointing to an endpoint like `https://mcp.api.coingecko.com/`, processes the request. The agent's underlying model determines that the CoinGecko MCP tool is needed to answer the query and calls it.
4.  The MCP tool fetches the relevant data (e.g., cryptocurrency prices) from the CoinGecko API.
5.  The agent receives the data back from the tool and formulates the final response, which is then printed.

In [ ]:
session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
response = await chat_with_agent("What's the current price of Bitcoin?",session.id)
print(f"Query: What's the current price of Bitcoin?")
print(f"Response: {response}")